# Register the ML Model to an Azure ML Workspace

In this notebookw, we will use MLFlow to track and register a machine learning model to an Azure Machine Learning Workspace. 

MLFlow provides a seamless way to log model parameters, metrics, and artifacts, and then register the model to Azure ML for deployment and further management. This ensures reproducibility, versioning, and easy access to the model for future use.


## Load the model and artifacts

In [1]:
import pickle

with open('Users/lubraz/reasoning-over-ml/train/artifacts/model.pkl', 'rb') as f:
    model = pickle.load(f)

with open('Users/lubraz/reasoning-over-ml/train/artifacts/X_test.pkl', 'rb') as f:
    X_test = pickle.load(f)

with open('Users/lubraz/reasoning-over-ml/train/artifacts/predictions.pkl', 'rb') as f:
    predictions = pickle.load(f)

with open('Users/lubraz/reasoning-over-ml/train/artifacts/results.pkl', 'rb') as f:
    results = pickle.load(f)

## Configure AML client

In [4]:
import os
from dotenv import load_dotenv
from azure.ai.ml import MLClient
from azure.identity import ManagedIdentityCredential

load_dotenv()

subscription_id = os.environ.get("AZURE_SUBSCRIPTION_ID")
resource_group = os.environ.get("AZURE_RESOURCE_GROUP")
workspace = os.environ.get("WORKSPACE_NAME")

credential = ManagedIdentityCredential(client_id=None)

ml_client = MLClient(credential, subscription_id, resource_group, workspace)

print("MLClient loaded successfully:", ml_client.workspace_name)

MLClient loaded successfully: wkmllab02


## Register the model to AML Workspace using MLFlow

In [ ]:
import mlflow
from mlflow.models.signature import infer_signature

signature = infer_signature(X_test, predictions)

# Set the new experiment (if it doesn't exist, MLflow will create it)
experiment_name = "XGBoost_Experiment"
mlflow.set_experiment(experiment_name)

with mlflow.start_run(run_name="XGBoost Model Run"):
    mlflow.log_metric("RMSLE", results['RMSLE'][0])
    mlflow.log_metric("RMSE", results['RMSE'][0])
    mlflow.log_metric("MSE", results['MSE'][0])
    mlflow.log_metric("MAE", results['MAE'][0])
    mlflow.log_metric("MAE_percent", results['MAE_percent'][0])
    

    mlflow.xgboost.log_model(model, artifact_path="xgb_model", signature=signature)

    
    print("XGBoost model and metrics successfully logged under experiment:", experiment_name)